In [1]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTConfig, SFTTrainer

In [2]:
dataset_path = 'data/eminem_lyrics_prompt_completion'
dataset = load_from_disk(dataset_path)

In [12]:
dataset = dataset.remove_columns('track_name')

In [24]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [25]:
training_args = SFTConfig(
    packing=False,
    output_dir='./results',  # Directory to save the model
    logging_dir='./logs',    # Directory to save logs
    logging_steps=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    fp16=True
)

In [26]:
def custom_formatting_func(example):
    prompt = example['prompt']
    completion = example['completion']
    text = prompt + completion
    return text

In [27]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    formatting_func=custom_formatting_func
)


Map:   0%|          | 0/1285 [00:00<?, ? examples/s]

In [28]:
trainer.train()

trainer.save_model('./trained_model')
trainer.save_state()

COMET INFO: An experiment with the same configuration options is already running and will be reused.


  0%|          | 0/963 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
# Load the trained model and tokenizer
model_path = './trained_model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Generate Rap Lyrics
prompt = "generate Rap lyrics like this lyrics:\nLook, I was gonna go easy on you not to hurt your feelings\nBut I'm only going to get this one chance "
inputs = tokenizer.encode(prompt, return_tensors="pt")
outputs = model.generate(inputs, max_length=200, do_sample=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

OSError: Incorrect path_or_model_id: './trained_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.